In [1]:
import numpy as np
from typing import Union
from collections import defaultdict

# Préparation des données
### Charger les données
def charger_donnees(nom_fichier: str, affichage: bool=False) -> str:
    """
    Charge les données du corpus à partir d'un fichier et les renvoie sous forme de liste de chaînes de caractères.

    Paramètres :
        nom_fichier : Nom du fichier.
        affichage : Si vrai, des informations sur les données seront affichées.
    
    Retour :
        str : Les données chargées sous forme de chaîne de caractères.
    """
    donnees = []

    with open(nom_fichier, "r", encoding='utf-8') as f:
        donnees = f.read()

    if affichage:
        print("Type de données :", type(donnees))
        print(f"Nombre de lettres : {len(donnees):,d}")
        print("Premières 100 lettres des données")
        print("-"*30)
        display(donnees[0:100])
        print("-"*30)

        print("Dernières 100 lettres des données")
        print("-"*30)
        display(donnees[-100:])
        print("-"*30)
        
    return donnees


In [2]:
def diviser_en_phrases(donnees: str) -> list[str]:
    """
    Divise les données par saut de ligne "\n"
    
    Paramètres :
        donnees (str) : Les données d'entrée sous forme de chaîne de caractères.
    
    Retourne :
        list[str] : Une liste de phrases.
    """
    phrases = donnees.split('\n')
    phrases = [phrase.strip() for phrase in phrases if phrase.strip()]
    
    return phrases


In [3]:
import re

def tokenizer_phrase(phrase: str) -> list[str]:
    """
    Tokenize une phrase en tokens.

    Paramètres :
        phrase (str) : La phrase d'entrée.
        
    Retourne :
        List[str] : Une liste de tokens.
    """
    phrase = phrase.lower()
    phrase = re.sub(r"[^\w\s]", "", phrase)
    tokens = phrase.split()
    return tokens


In [4]:
def obtenir_donnees_tokenisees(donnees: str) -> tuple[list[str], list[list[str]]]:
    """
    Tokenise les données en phrases et en mots.

    Paramètres :
        donnees (str) : Les données d'entrée sous forme de chaîne de caractères.

    Retourne :
        tuple[list[str], list[list[str]]] : Un tuple contenant la liste des phrases et la liste des phrases tokenisées.
    """
    phrases = diviser_en_phrases(donnees)
    
    phrases_tokenisees = [tokenizer_phrase(p) for p in phrases]
    
    return phrases, phrases_tokenisees


In [5]:
def obtenir_vocabulaire(donnees_tokenisees: list[list[str]]) -> set[str]:
    """
    Extrait le vocabulaire à partir d'une liste de données tokenisées.

    Paramètres :
        donnees_tokenisees (list[list[str]]) : Une liste de phrases tokenisées.

    Retourne :
        set[str] : Un ensemble contenant les mots uniques présents dans les données tokenisées.
    """
    vocabulaire = set()
    for sous_liste in donnees_tokenisees:
        for element in sous_liste:
            vocabulaire.add(element)
    return vocabulaire


In [6]:
def diviser_donnees(donnees_tokenisees: list[list[str]], ratio_entrainement: float=0.8,
                affichage: bool=False) -> tuple[list[list[str]], list[list[str]]]:
    """
    Divise les données en ensembles d'entraînement et de test en fonction du ratio spécifié.
    
    Paramètres :
        donnees_tokenisees (list[list[str]]) : Les données d'entrée sous forme d'une liste de listes de tokens.
        ratio_entrainement (float) : Le ratio des données à utiliser pour l'entraînement (entre 0 et 1).
        affichage (bool) : Si vrai, des informations sur la division seront affichées.
        
    Retourne :
        Tuple[List[list[str]], List[list[str]]] : Les ensembles d'entraînement et de test sous forme de listes de phrases ou de séquences.
    """
    taille_entrainement = int(len(donnees_tokenisees) * ratio_entrainement)
    donnees_entrainement = donnees_tokenisees[:taille_entrainement]
    donnees_test = donnees_tokenisees[taille_entrainement:]

    if affichage:
        print(f"{len(donnees_tokenisees)} données sont divisées en {len(donnees_entrainement)} ensemble d'entraînement et {len(donnees_test)} ensemble de test")

        print("Premier échantillon d'entraînement :")
        print(donnees_entrainement[0])
            
        print("Premier échantillon de test :")
        print(donnees_test[0])
    
    return donnees_entrainement, donnees_test


In [7]:
def est_mot(mot: str, vocabulaire: set[str]) -> bool:
    """
    Vérifie si un mot est présent dans le vocabulaire.

    Paramètres :
        mot (str) : Le mot à vérifier.
        vocabulaire (set[str]) : L'ensemble de mots représentant le vocabulaire.

    Retourne :
        bool : True si le mot est dans le vocabulaire, False sinon.
    """
    return mot.lower() in vocabulaire


In [8]:
def calculer_distance_edition(source: str, cible: str, cout_insertion: int=1, 
                            cout_suppression: int=1, cout_remplacement: int=2) -> tuple[np.ndarray, int]:
    """
    Calcule la distance d'édition minimale entre deux mots.
    
    Paramètres :
        source (str) : Le mot source.
        cible (str) : Le mot cible.
        cout_insertion (int) : Le coût de l'insertion (par défaut 1).
        cout_suppression (int) : Le coût de la suppression (par défaut 1).
        cout_remplacement (i nt) : Le coût du remplacement (par défaut 2).

    Retourne :
        D : une matrice de dimensions len(source)+1 par len(cible)+1 contenant les distances d'édition minimales
        med : la distance d'édition minimale (med) nécessaire pour convertir la chaîne source en la cible
    """
    m, n = len(source), len(cible)
    D = np.zeros((m+1, n+1), dtype=int)
    
    for ligne in range(1, m+1):
        D[ligne, 0] = D[ligne-1, 0] + cout_suppression
        
    for colonne in range(1, n+1):
        D[0, colonne] = D[0, colonne-1] + cout_insertion
        
    for ligne in range(1, m+1):
        for colonne in range(1, n+1):
            cout_rempl = cout_remplacement
            if source[ligne-1] == cible[colonne-1]:
                cout_rempl = 0
            D[ligne, colonne] = min(D[ligne-1, colonne] + cout_suppression, D[ligne, colonne-1] + cout_insertion, D[ligne-1, colonne-1] + cout_rempl)
          
    med = D[m, n]
    
    return D, med


In [9]:
def corrections_une_modification(mot: str, vocabulaire: list[str]) -> set[str]:
    """
    Génère un ensemble de corrections possibles pour un mot mal orthographié avec une seule modification.

    Paramètres :
        mot (str) : Le mot mal orthographié.
        vocabulaire (list[str]) : Une liste de mots du vocabulaire.

    Retourne :
        set[str] : Un ensemble de corrections possibles.
    """

    alphabet = 'abcdefghijklmnopqrstuvwxyz'
    decoupes = [(mot[:i], mot[i:]) for i in range(len(mot) + 1)]

    suppressions = [gauche + droite[1:] for gauche, droite in decoupes if droite]
    suppressions = [s for s in suppressions if est_mot(s, vocabulaire)]

    insertions = [gauche + c + droite for gauche, droite in decoupes for c in alphabet]
    insertions = [i for i in insertions if est_mot(i, vocabulaire)]

    remplacements = [gauche + c + droite[1:] for gauche, droite in decoupes if droite for c in alphabet]
    remplacements = [r for r in remplacements if est_mot(r, vocabulaire)]

    transpositions = [gauche + droite[1] + droite[0] + droite[2:] for gauche, droite in decoupes if len(droite) > 1]
    transpositions = [t for t in transpositions if est_mot(t, vocabulaire)]
    
    return set(suppressions + insertions + remplacements + transpositions)


In [10]:
def obtenir_corrections(mot: str, vocabulaire: list[str], n_modifications: int=1, 
                        distance_max: int=2) -> set[str]:
    """
    Génère une liste de corrections possibles pour un mot mal orthographié en fonction du vocabulaire donné et de la distance d'édition maximale.

    Paramètres :
        mot (str) : Le mot mal orthographié.
        vocabulaire (list[str]) : Une liste de mots du vocabulaire.
        n_modifications (int) : Le nombre de modifications autorisées pour générer des corrections (par défaut 1).
        distance_max (int) : La distance d'édition maximale autorisée pour qu'une correction soit considérée (par défaut 2).

    Retourne :
        corrections_possibles (set[str]) : Un ensemble de corrections possibles.
    """
    corrections_possibles = set()

    if n_modifications == 1:
        corrections_possibles = {corr for corr in corrections_une_modification(mot, vocabulaire) if calculer_distance_edition(mot, corr)[1] <= distance_max}
    else:
        modifications_precedentes = {mot}
        for _ in range(n_modifications):
            modifications_actuelles = set()
            for modification_precedente in modifications_precedentes:
                nouvelles_corrections = {corr for corr in corrections_une_modification(modification_precedente, vocabulaire) if calculer_distance_edition(mot, corr)[1] <= distance_max}
                modifications_actuelles.update(nouvelles_corrections)
            modifications_precedentes = modifications_actuelles
        corrections_possibles = modifications_precedentes

    return corrections_possibles


In [11]:
def compter_mots(phrases_tokenisees: list[list[str]]) -> dict[str, int]:
    """
    Compte le nombre d'apparitions de chaque mot dans les phrases tokenisées.

    Paramètres :
        sentences_tokenises (list[list[str]]) : Liste de listes de chaînes de caractères.

    Retourne :
        word_counts (dict[str, int]) : Dictionnaire qui fait correspondre chaque mot (str) à sa fréquence (int).
    """

    word_counts = {}
    for phrase in phrases_tokenisees:
        for token in phrase:
            if token not in word_counts.keys():
                word_counts[token] = 1
            else:
                word_counts[token] += 1

    return word_counts


In [12]:
def obtenir_mots_avec_frequence_superieure_ou_egale(phrases_tokenisees: list[list[str]], 
                                   seuil_frequence: int) -> list[str]:
    """
    Trouve les mots qui apparaissent N fois ou plus.

    Paramètres :
        tokenized_sentences (list[list[str]]) : Liste de listes de phrases.
        seuil_frequence (int) : Nombre minimum d'occurrences pour qu'un mot fasse partie du vocabulaire restreint.

    Retourne :
        closed_vocab (list[str]) : Liste des mots qui apparaissent N fois ou plus.
    """
    closed_vocab = []
    word_counts = compter_mots(phrases_tokenisees)
    for word, cnt in word_counts.items():
        if cnt >= seuil_frequence:
            closed_vocab.append(word)

    return closed_vocab


In [13]:
def remplacer_mots_inconnus_par_unk(phrases_tokenisees: list[list[str]], 
                             vocabulaire: list[str], unknown_token: str="<unk>"
                             ) -> list[list[str]]:
    """
    Replace words not in the given vocabulary with the unknown token.

    Parameters:
        phrases_tokenisees: List of lists of strings
        vocabulaire: List of strings that we will use
        unknown_token: A string representing unknown (out-of-vocabulary) words
    
    Returns:
        replaced_phrases_tokenisees: List of lists of strings, with words not in the vocabulary replaced
    """
    
    vocabulaire = set(vocabulaire)
    replaced_phrases_tokenisees = []
    for phrase in phrases_tokenisees:
        replaced_phrase = []
        for token in sentence:
            if token in vocabulaire:
                replaced_phrase.append(token)
            else:
                replaced_phrase.append(unknown_token)
        replaced_phrases_tokenisees.append(replaced_phrase)
        
    return replaced_phrases_tokenisees


In [14]:
def données_avec_unk(données: list[list[str]], seuil_frequence: int):
    """
    Prétraite les données en remplaçant les mots peu fréquents par le jeton inconnu.
    
    Paramètres :
        données (list[list[str]]) : Liste de listes de chaînes de caractères.
        freq_threshold (int) : Les mots dont le compte est inférieur à cette valeur sont considérés comme inconnus.

    Retourne :
        Tuple de
        - données avec les mots peu fréquents remplacés par "<unk>"
        - vocabulaire des mots qui apparaissent au moins n fois dans les données d'entraînement
    """
   
    vocabulaire = obtenir_mots_avec_frequence_superieure_ou_egale(phrases_tokenisees, seuil_frequence)
    données_remplacées = remplacer_mots_inconnus_par_unk(phrases_tokenisees, vocabulaire)
    
    return données_remplacées, vocabulaire
fd

In [15]:
def compter_n_grammes(données: list[list[str]], n: int=2, 
                      jeton_début: str='<d>', jeton_fin: str= '<f>') -> dict:
    """
    Compte tous les n-grammes dans les données fournies.
    
    Paramètres :
        données (list[list[str]]) : Liste de listes de mots.
        n (int) : nombre de mots dans une séquence (par défaut 2).
        jeton_début (str) : une chaîne de caractères indiquant le début de la phrase (par défaut '<d>').
        jeton_fin (str) : une chaîne de caractères indiquant la fin de la phrase (par défaut '<f>').
    
    Retourne :
        n_grammes (dict) : Un dictionnaire qui mappe un tuple de n mots à sa fréquence.
    """
    n_grammes = {}

    for phrase in données:
        phrase = [jeton_début]*n + phrase + [jeton_fin]
        phrase = tuple(phrase)
        m = len(phrase) if n == 1 else len(phrase) - 1
        for i in range(m): 
            n_gramme = phrase[i:i+n]
            if n_gramme in n_grammes.keys():
                n_grammes[n_gramme] += 1
            else:
                n_grammes[n_gramme] = 1
    
    return n_grammes


In [16]:
def estimer_probabilite(mot: str, n_gramme_precedent: list[str], comptes_n_grammes: dict, 
                       comptes_n_plus1_grammes: dict, taille_vocabulaire: int, k: float=1.0) -> float:
    """
    Estime les probabilités d'un prochain mot en utilisant les comptes de n-grammes avec lissage k.
    
    Paramètres :
        mot (str) : Prochain mot.
        n_gramme_precedent (list[str]) : Une séquence de mots de longueur n.
        comptes_n_grammes (dict) : Dictionnaire des comptes des n-grammes.
        comptes_n_plus1_grammes (dict) : Dictionnaire des comptes des (n+1)-grammes.
        taille_vocabulaire (int) : Nombre de mots dans le vocabulaire.
        k (float) : Constante positive, paramètre de lissage (par défaut 1.0).
    
    Retourne :
        probabilité (float) : Une probabilité.
    """
    n_gramme_precedent = tuple(n_gramme_precedent)
    comptage_n_gramme_precedent = comptes_n_grammes[n_gramme_precedent] if n_gramme_precedent in comptes_n_grammes else 0
    dénominateur = comptage_n_gramme_precedent + k * taille_vocabulaire

    n_plus1_gramme = n_gramme_precedent + (mot,)
    comptage_n_plus1_gramme = comptes_n_plus1_grammes[n_plus1_gramme] if n_plus1_gramme in comptes_n_plus1_grammes else 0
    numérateur = comptage_n_plus1_gramme + k
    
    probabilité = numérateur / dénominateur
        
    return probabilité


In [17]:
def correction_orthographe(texte: str, vocabulaire: set[str], top_n: int=2, n_g: int=2, k: int=1.0,
                          n_edits: int=1, max_distance: int=2) -> tuple[dict, str]:
    """
    Effectue une correction orthographique sur le texte donné en utilisant un modèle de langue n-gramme.

    Paramètres :
        texte (str) : Le texte sur lequel effectuer la correction orthographique.
        vocabulaire (set[str]) : Un ensemble de mots représentant le vocabulaire.
        top_n (int) : Le nombre de suggestions les plus probables à prendre en compte (par défaut 2).
        n_g (int) : L'ordre du modèle de langue n-gramme (par défaut 2).
        k (int) : Constante positive, paramètre de lissage (par défaut 1.0).
        n_edits (int) : Le nombre maximum de modifications autorisées dans une correction suggérée (par défaut 1).
        max_distance (int) : La distance maximale de modification autorisée pour une correction suggérée (par défaut 2).

    Retourne :
        sorted_dict (dict) : Dictionnaire de suggestions.
        texte_corrige (str) : La version corrigée du texte d'entrée.
    """
    n_meilleures = []
    suggestions = dict()
    phrases, phrases_tokenisees = obtenir_donnees_tokenisees(texte)
    n_grammes = compter_n_grammes(phrases_tokenisees, n_g)
    n_plus1_grammes = compter_n_grammes(phrases_tokenisees, n_g+1)
    texte_corrige = texte
    
    for phrase in phrases_tokenisees:
        index = None
        phrase_tmp = ['<d>']*n_g + phrase + ['<f>']
        
        for token in phrase:
            probas = dict()
            if not est_mot(token, vocabulaire):
                index = phrase_tmp.index(token)
                n_gramme_precedent = tuple(phrase_tmp[abs(index-2):index])
                comptes_n_grammes = n_grammes[n_gramme_precedent]
                corrections = obtenir_corrections(token, vocabulaire, n_edits, max_distance)
                corrections = [c for c in corrections if est_mot(c, vocabulaire)]
                for corr in corrections:
                    proba = estimer_probabilite(corr, n_gramme_precedent, n_grammes,
                                               n_plus1_grammes, len(vocabulaire), k)
                    probas[corr] = proba
                suggestions[token] = probas
        
        suggestions_triees = {k: dict(sorted(v.items(), key=lambda item: item[1], reverse=True)) for k, v in suggestions.items()}
        sorted_dict = {}
        for cle, dict_interne in suggestions_triees.items():
            dict_interne_trie = dict(sorted(dict_interne.items(), key=lambda item: item[1], reverse=True)[:top_n])
            sorted_dict[cle] = dict_interne_trie

        for cle in sorted_dict.keys():
            if cle in texte_corrige:
                premier_mot_interne = next(iter(sorted_dict[cle]))
                texte_corrige = texte_corrige.replace(cle, premier_mot_interne)

    return sorted_dict, texte_corrige

print("Terminé !")


Terminé !


In [20]:
donnees = charger_donnees('corpus.txt', affichage=True)
phrases, donnees_tokenisees = obtenir_donnees_tokenisees(donnees)
donnees_entrainement, donnees_test = diviser_donnees(donnees_tokenisees, 0.8)
vocabulaire = obtenir_vocabulaire(donnees_tokenisees)

n_gr = 2
n_grams=compter_n_grammes(donnees_tokenisees, n_gr)
n1_grams=compter_n_grammes(donnees_tokenisees, n_gr+1)

text = "i am enjaying time.\nI actually likod Derek Morris as a Ranger."

sorted_dict, texte_corrige = correction_orthographe(text, vocabulaire)

print(f"Originale texte:\n{text}\n")
print(f"Corrige texte:\n{texte_corrige}\n")
print(f"The misspelled words and thier corrections:")
for mot in sorted_dict.keys():
    print('-'*50)
    print(f"  {mot}:")
    for c, p in sorted_dict[mot].items():
        print(f"{c}:\t{p}")

Type de données : <class 'str'>
Nombre de lettres : 6,488,665
Premières 100 lettres des données
------------------------------


'The Project Gutenberg EBook of The Adventures of Sherlock Holmes\nby Sir Arthur Conan Doyle\n(#15 in o'

------------------------------
Dernières 100 lettres des données
------------------------------


' i, c, ord(c), big[max(0, i-10):min(N, i+10)]\n        s.add(c)\n  print s\n  print [ord(c) for c in s]'

------------------------------
Originale texte:
i am enjaying time.
I actually likod Derek Morris as a Ranger.

Corrige texte:
i am enjoying time.
I actually liked Derek Morris as a Ranger.

The misspelled words and thier corrections:
--------------------------------------------------
  enjaying:
enjoying:	2.531837861103375e-05
--------------------------------------------------
  likod:
liked:	2.531837861103375e-05
--------------------------------------------------
  derek:
dere:	2.531837861103375e-05


In [27]:
!pip install pyenchant



     ---------------------------------------- 11.9/11.9 MB 2.0 MB/s eta 0:00:00


In [31]:
import collections
import enchant
import itertools
import math
import nltk.data
import nltk
nltk.download('punkt')
import re

class Sentence_Corrector:
    def __init__(self, training_file):
        self.laplaceUnigramCounts = collections.defaultdict(lambda: 0)
        self.laplaceBigramCounts = collections.defaultdict(lambda: 0)
        self.total = 0
        self.sentences = []
        self.importantKeywords = set()
        self.d = enchant.Dict("en_US")
        self.tokenize_file(training_file)
        self.train()

    def tokenize_file(self, file):
        tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
        with open(file) as f:
            content = f.read()
        for sentence in tokenizer.tokenize(content):
            sentence_clean = [i.lower() for i in re.split('[^a-zA-Z]+', sentence) if i]
            self.sentences.append(sentence_clean)

    def train(self):
        for sentence in self.sentences:
            sentence.insert(0, '<s>')
            sentence.append('</s>')
            for i in range(len(sentence) - 1):
                token1 = sentence[i]
                token2 = sentence[i + 1]
                self.laplaceUnigramCounts[token1] += 1
                self.laplaceBigramCounts[(token1, token2)] += 1
                self.total += 1
            self.total += 1
            self.laplaceUnigramCounts[sentence[-1]] += 1

    def candidate_word(self, word):
        suggests = []
        for candidate in self.importantKeywords:
            if candidate.startswith(word):
                suggests.append(candidate)
        suggests.append(word)

        if len(suggests) == 1:
            suggests = self.d.suggest(word)
            suggests = [suggest.lower() for suggest in suggests][:4]
            suggests.append(word)
            suggests = list(set(suggests))

        return suggests, len(suggests)

    def candidate_sentence(self, sentence):
        candidate_sentences = []
        words_count = {}
        for word in sentence:
            candidate_sentences.append(self.candidate_word(word)[0])
            words_count[word] = self.candidate_word(word)[1]

        candidate_sentences = list(itertools.product(*candidate_sentences))
        return candidate_sentences, words_count

    def correction_score(self, words_count, old_sentence, new_sentence):
        score = 1
        for i in range(len(new_sentence)):
            if new_sentence[i] in words_count:
                score *= 0.95
            else:
                score *= (0.05 / (words_count[old_sentence[i]] - 1))
        return math.log(score)

    def noisy_correction_score(self, words_count, old_sentence, new_sentence, error_probability):
        score = 1
        for i in range(len(new_sentence)):
            if new_sentence[i] in words_count:
                score *= (1 - error_probability)
            else:
                score *= (error_probability / (words_count[old_sentence[i]] - 1))
        return math.log(score)

    def score(self, sentence):
        score = 0.0
        for i in range(len(sentence) - 1):
            if self.laplaceBigramCounts[(sentence[i], sentence[i + 1])] > 0:
                score += math.log(self.laplaceBigramCounts[(sentence[i], sentence[i + 1])])
                score -= math.log(self.laplaceUnigramCounts[sentence[i]])
            else:
                score += (math.log(self.laplaceUnigramCounts[sentence[i + 1]] + 1) + math.log(0.4))
                score -= math.log(self.total + len(self.laplaceUnigramCounts))
        return score

    def return_best_sentence(self, old_sentence):
        bestScore = float('-inf')
        bestSentence = []
        old_sentence = [word.lower() for word in old_sentence.split()]
        sentences, word_count = self.candidate_sentence(old_sentence)
        for new_sentence in sentences:
            new_sentence = list(new_sentence)
            score = self.noisy_correction_score(word_count, new_sentence, old_sentence, 0.1)  # Error probability of 0.1
            new_sentence.insert(0, '<s>')
            new_sentence.append('</s>')
            score += self.score(new_sentence)
            if score >= bestScore:
                bestScore = score
                bestSentence = new_sentence
        bestSentence = ' '.join(bestSentence[1:-1])
        return bestSentence, bestScore


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\pc\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.


In [32]:
corrector = Sentence_Corrector('corpus.txt')
#corrector.return_best_sentence('this is wron spallin word')
corrector.return_best_sentence('aoccdrning to a resarch at cmabridge university')
#corrector.return_best_sentence('it does not mttaer in waht oredr the ltteers')
#corrector.return_best_sentence('the olny important tihng is taht')
#corrector.return_best_sentence('Can they leav him my messages')
#corrector.return_best_sentence('This used to belong to thew queen')

('according to a research at cambridge university', -53.411284186487)